# Slit Width Trade Book
### Bandpass vs. SNR vs. Along-Track Spatial Resolution

In [2]:
# stdlib
import math
from pathlib import Path

# external
import astropy.units as unit
import numpy as np
import pandas as pd
import plot
from IPython.display import display

# project
from architect import luts
from architect.libs import utillib
from architect.systems.optical import (
    diffractors,
    foreoptics,
    lenses,
    masks,
    sensors,
    spectrometers,
)

## Setup
Run this section onwards when a parameter is updated.

### Parameters

In [14]:
# constants
target_wavelength = 1300 * unit.nm
lower_wavelength = 900 * unit.nm
upper_wavelength = 1700 * unit.nm
beam_diameter = 12 * unit.mm
fringe_frequency = 400 / unit.mm
skew_angle = np.array([0, 0]) * unit.deg
orbital_altitude = 505 * unit.km
incident_angle = 36.3 * unit.deg
spectral_order = 1

# variables
widths = np.arange(start=10, stop=100, step=1) * unit.micrometer

### Instantiation

In [17]:
# components
sensor = sensors.TauSWIR()
foreoptic = foreoptics.Foreoptic(
    focal_length=50 * unit.mm, diameter=35 * unit.mm, image_diameter=10 * unit.mm
)
diffractor = diffractors.TransmissiveDiffractor(fringe_frequency=fringe_frequency)
collimator = lenses.Lens(focal_length=13 * unit.mm)
radiance = luts.load("atmosphere/radiance_min")

## Pipeline
The computational graph

In [18]:
bandpass = []
snr = []
spatial_resolution = []

for width in widths:
    # Modify the slit width and then reinstantiate the HyperspectralImager
    slit = masks.RectSlit(size=(width, 7.68 * unit.mm))
    payload = spectrometers.HyperspectralImager(
        sensor=sensor,
        foreoptic=foreoptic,
        slit=slit,
        diffractor=diffractor,
        collimator=collimator,
    )

    # Determine the bandpass with the new slit width
    bandpass_measurement = payload.get_bandpass(
        width, incident_angle, spectral_order
    )  # .to(unit.nm)
    bandpass.append(bandpass_measurement.value)

    # Determine the SNR for the new slit width
    snr_measurment = payload.get_signal_to_noise(
        radiance=radiance, wavelength=target_wavelength
    )
    snr.append(snr_measurment.value)

    # Determine the Spatial Resolution for the new slit
    spatial_resolution_measurment = payload.get_spatial_resolution(
        wavelength=target_wavelength,
        target_distance=orbital_altitude,
        skew_angle=skew_angle[0],
    )
    spatial_resolution.append(spatial_resolution_measurment.to(unit.km).value)

## Plots

In [20]:
print(spatial_resolution)
widths_label = "Slit Width (microns)"
bandpass_label = "Spectral Bandpass (nm)"
snr_label = "SNR"
spatial_resolution_label = "Spatial Resolution (km)"
data = {
    widths_label: widths[:],
    bandpass_label: bandpass[:],
    snr_label: snr[:],
    spatial_resolution_label: spatial_resolution[:],
}

df = pd.DataFrame.from_dict(data=data)
fig = plot.line(
    df=df,
    x=widths_label,
    y=[bandpass_label, snr_label, spatial_resolution_label],
    title="Slit Width Trade Off",
    dark=True,
)
fig.show()
display(df)

[0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515, 0.1515]


,Slit Width (microns),Spectral Bandpass (nm),SNR,Spatial Resolution (km)
0,10.0,64.474263,2.011972,0.1515
1,11.0,70.921689,2.213160,0.1515
2,12.0,77.369115,2.414346,0.1515
3,13.0,83.816541,2.615530,0.1515
4,14.0,90.263968,2.816712,0.1515
...,...,...,...,...
85,95.0,612.505495,19.106736,0.1515
86,96.0,618.952921,19.307777,0.1515
87,97.0,625.400347,19.508816,0.1515
88,98.0,631.847773,19.709852,0.1515
